<a href="https://colab.research.google.com/github/Manasa2810/Label_studio/blob/main/Task3/stt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy pandas scikit-learn statsmodels


In [5]:
import json
import pandas as pd
from sklearn.metrics import cohen_kappa_score

# File paths
annotator1_file = "/content/anno1_nlp.csv"
annotator2_file = "/content/ann2_nlp.csv"

# Loading data
annotator1_df = pd.read_csv(annotator1_file, header=None)
annotator2_df = pd.read_csv(annotator2_file, header=None)


annotator1_df.columns = ['annotation_id', 'annotator', 'created_at', 'id', 'lead_time', 'pos_tags', 'text', 'updated_at']
annotator2_df.columns = ['annotation_id', 'annotator', 'created_at', 'id', 'label', 'lead_time', 'text', 'updated_at']


def safe_json_loads(s):
    try:
        return json.loads(s)
    except (json.JSONDecodeError, TypeError):
        return []

# Parse annotations
annotator1_df['parsed_pos_tags'] = annotator1_df['pos_tags'].apply(safe_json_loads)
annotator2_df['parsed_label'] = annotator2_df['label'].apply(safe_json_loads)

# Flatten and align annotations
annotator1_labels = []
annotator2_labels = []

for i in range(len(annotator1_df)):
    a1_tokens = annotator1_df['parsed_pos_tags'].iloc[i]
    a2_tokens = annotator2_df['parsed_label'].iloc[i]

    # Create dictionaries for alignment
    a1_dict = {f"{token['start']}_{token['end']}": token['labels'][0].strip('"') for token in a1_tokens}
    a2_dict = {f"{token['start']}_{token['end']}": token['labels'][0].strip('"') for token in a2_tokens}

    # Align and compare tokens
    for position, label in a1_dict.items():
        if position in a2_dict:
            annotator1_labels.append(label)
            annotator2_labels.append(a2_dict[position])

# Check for empty labels
if not annotator1_labels or not annotator2_labels:
    print("Error: No aligned labels found. Please check your data!")
else:
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(annotator1_labels, annotator2_labels)

    # Interpreting Cohen's Kappa
    def interpret_kappa(kappa_score):
        if kappa_score < 0:
            return "Poor agreement."
        elif 0.01 <= kappa_score <= 0.20:
            return "Slight agreement."
        elif 0.21 <= kappa_score <= 0.40:
            return "Fair agreement."
        elif 0.41 <= kappa_score <= 0.60:
            return "Moderate agreement."
        elif 0.61 <= kappa_score <= 0.80:
            return "Substantial agreement."
        elif 0.81 <= kappa_score <= 1.00:
            return "Almost perfect agreement."
        else:
            return "Invalid value."


    print(f"Cohen's Kappa: {kappa}")
    print(f"Interpretation: {interpret_kappa(kappa)}")


Cohen's Kappa: 0.8399662346341206
Interpretation: Almost perfect agreement.


In [9]:
import pandas as pd

# Load the annotator 3 CSV file
ann3_df = pd.read_csv('/content/ann3.csv')

# Start new annotation IDs from the desired number (e.g., 60)
starting_id = 21
ann3_df['annotation_id'] = range(starting_id, starting_id + len(ann3_df))

# Save the updated annotator 3 file
ann3_df.to_csv('updated_ann3_file.csv', index=False)

print("Updated annotation IDs saved in 'updated_ann3_file.csv'")


Updated annotation IDs saved in 'updated_ann3_file.csv'


In [10]:
import pandas as pd
from statsmodels.stats.inter_rater import fleiss_kappa

# File paths for the three annotators' data
file1 = "/content/ann1_clp.csv"  # Replace with the path to annotator 1's file
file2 = "/content/ann2_cv.csv"  # Replace with the path to annotator 2's file
file3 = "/content/updated_ann3_file.csv"  # Replace with the path to annotator 3's file

# Load data
annotator1_df = pd.read_csv(file1)
annotator2_df = pd.read_csv(file2)
annotator3_df = pd.read_csv(file3)

# Merge datasets on 'annotation_id' (assuming this column exists in all files)
merged_df = annotator1_df[['annotation_id', 'choice']].rename(columns={'choice': 'annotator1'}).merge(
    annotator2_df[['annotation_id', 'choice']].rename(columns={'choice': 'annotator2'}),
    on='annotation_id'
).merge(
    annotator3_df[['annotation_id', 'choice']].rename(columns={'choice': 'annotator3'}),
    on='annotation_id'
)

# Categories and build category matrix
categories = pd.concat([merged_df['annotator1'], merged_df['annotator2'], merged_df['annotator3']]).unique()
category_matrix = []

for _, row in merged_df.iterrows():
    # Count occurrences of each category for the current item
    counts = pd.Series([row['annotator1'], row['annotator2'], row['annotator3']]).value_counts()
    row_matrix = [counts.get(category, 0) for category in categories]
    category_matrix.append(row_matrix)

# Convert to DataFrame for easier debugging
category_matrix_df = pd.DataFrame(category_matrix, columns=categories)
print("Category Matrix:")
print(category_matrix)
print(category_matrix_df)

# Calculate Fleiss' Kappa
kappa = fleiss_kappa(category_matrix, method='fleiss')

# Interpretation
def interpret_kappa(kappa_score):
    if kappa_score < 0:
        return "Poor agreement."
    elif 0.01 <= kappa_score <= 0.20:
        return "Slight agreement."
    elif 0.21 <= kappa_score <= 0.40:
        return "Fair agreement."
    elif 0.41 <= kappa_score <= 0.60:
        return "Moderate agreement."
    elif 0.61 <= kappa_score <= 0.80:
        return "Substantial agreement."
    elif 0.81 <= kappa_score <= 1.00:
        return "Almost perfect agreement."
    else:
        return "Invalid value."

# Output Fleiss' Kappa and interpretation
print(f"Fleiss' Kappa: {kappa}")
print(f"Interpretation: {interpret_kappa(kappa)}")


Category Matrix:
[[1, 2], [3, 0], [0, 3], [0, 3], [0, 3], [2, 1], [1, 2], [1, 2], [1, 2], [3, 0], [3, 0], [1, 2], [2, 1], [0, 3], [3, 0], [0, 3], [0, 3], [2, 1], [3, 0], [3, 0]]
    Trucks  No Trucks
0        1          2
1        3          0
2        0          3
3        0          3
4        0          3
5        2          1
6        1          2
7        1          2
8        1          2
9        3          0
10       3          0
11       1          2
12       2          1
13       0          3
14       3          0
15       0          3
16       0          3
17       2          1
18       3          0
19       3          0
Fleiss' Kappa: 0.4660734149054503
Interpretation: Moderate agreement.


In [11]:
import pandas as pd
from statsmodels.stats.inter_rater import fleiss_kappa

# File paths for the two annotators' data
file1 = "/content/ann1_clp.csv"  # Replace with the path to annotator 1's file
file2 = "/content/ann2_cv.csv"  # Replace with the path to annotator 2's file

# Load data
annotator1_df = pd.read_csv(file1)
annotator2_df = pd.read_csv(file2)

# Merge datasets on 'annotation_id' (assuming this column exists in both files)
merged_df = annotator1_df[['annotation_id', 'choice']].rename(columns={'choice': 'annotator1'}).merge(
    annotator2_df[['annotation_id', 'choice']].rename(columns={'choice': 'annotator2'}),
    on='annotation_id'
)

# Extract unique categories
categories = pd.concat([merged_df['annotator1'], merged_df['annotator2']]).unique()

# Build category matrix
category_matrix = []

for _, row in merged_df.iterrows():
    # Count occurrences of each category for the current item
    counts = pd.Series([row['annotator1'], row['annotator2']]).value_counts()
    row_matrix = [counts.get(category, 0) for category in categories]
    category_matrix.append(row_matrix)

# Convert to DataFrame for easier debugging
category_matrix_df = pd.DataFrame(category_matrix, columns=categories)
print("Category Matrix:")
print(category_matrix_df)

# Calculate Fleiss' Kappa
kappa = fleiss_kappa(category_matrix, method='fleiss')

# Interpretation
def interpret_kappa(kappa_score):
    if kappa_score < 0:
        return "Poor agreement."
    elif 0.01 <= kappa_score <= 0.20:
        return "Slight agreement."
    elif 0.21 <= kappa_score <= 0.40:
        return "Fair agreement."
    elif 0.41 <= kappa_score <= 0.60:
        return "Moderate agreement."
    elif 0.61 <= kappa_score <= 0.80:
        return "Substantial agreement."
    elif 0.81 <= kappa_score <= 1.00:
        return "Almost perfect agreement."
    else:
        return "Invalid value."

# Output Fleiss' Kappa and interpretation
print(f"Fleiss' Kappa: {kappa}")
print(f"Interpretation: {interpret_kappa(kappa)}")


Category Matrix:
    Trucks  No Trucks
0        1          1
1        2          0
2        0          2
3        0          2
4        0          2
5        1          1
6        1          1
7        0          2
8        0          2
9        2          0
10       2          0
11       1          1
12       1          1
13       0          2
14       2          0
15       0          2
16       0          2
17       1          1
18       2          0
19       2          0
Fleiss' Kappa: 0.3939393939393937
Interpretation: Fair agreement.
